In [1]:
from scipy.stats import hypergeom

# Example of validation bias detection

- A gene prioritization model was evaluated on a validation set consisting of $50$ genes. 


- The model's $recall@150$, which measures its ability to identify relevant genes, was found to be $0.6$. In simpler terms, when considering the top $150$ ranked genes, $30$ of them were actually part of the validation set.


- The model was tested on a very polygenic disease - cancer. Cancer Gene Census currently contains 579 genes - therefore, we can safely assume that there are at least $400$ genes that are causal to cancer.

Then, we can use hypergeometric distribution to find whether the model's performance of $recall@150 = 0.6$ is attainable without violating the selected completely at random (SCAR) assumption.

In [2]:
M = 400 # assumed number of disease genes
N = 50 # number of genes in validation set
k = 150 # only performance of top-k ranked genes in considered 
tp = 30 # number of disease genes in top-k

Under the assumption of the perfect model and the SCAR assumption, the generative process of the number of true positives (**TP**) can be described as follows: (i) there are $M$ disease genes in total, (ii) $k$ of them are top-$k$ ranked because we use the perfect model, (iii) $N$ of the disease genes are sampled into a validation set. Then, **TP** indicates how many of the sampled genes are in the top-$k$. Using this formulation, we can notice that the number of **TP** follows a hypergeometric distribution with the total population of $M$; $k$ objects of interest and $N$ draws.

In [3]:
print(f'P-value is {1 - hypergeom.cdf(tp-1, M, k, N):.2e}')

P-value is 4.77e-04


Thus, we can reject the SCAR assumption if there are at least 400 genes that cause the disease. 

# Identifying the minimum assumption about the number of disease genes

Previously, we concluded that it is safe to assume that there are at least 400 genes causal to cancer. However, maybe we can relax this assumption even more while still having the statistical power to reject the SCAR assumption. 

In [5]:
def recall_at_k_pval(k, N, tp):
    for M_lb in range(tp, 20000):
        if 1 - hypergeom.cdf(tp-1, M_lb, k, N) < 0.05:
            return M_lb

recall_at_k_pval(k, N, tp)

311

As we see, it was enough to assume that there are 311 genes to detect validation bias.

In [6]:
print(f'P-value for M = 311: {1 - hypergeom.cdf(tp-1, 311, k, N):.2e}')

P-value for M = 311: 4.80e-02
